# Street View House Numbers - Evaluation on Test dataset

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import svhn
import graphics
import keras_utils
from keras.utils import np_utils
import keras
from keras import backend as K

%matplotlib inline

max_digits = 7
image_size = (54,128)

# print the keras version used
import keras
print "Keras version : {}".format(keras.__version__)

Using TensorFlow backend.


Keras version : 1.2.0


## Load test dataset
Here we load the sample points as individual prediction targets. We will not flatten the data like we did during training, since we have to predict the complete sequence of digits in an image together.

In [2]:
# load the data file (takes time)
rawdata = svhn.read_process_h5('../inputs/test/digitStruct.mat')

In [3]:
# extract resized images, counts and label sequences for each sample
def generateTestData(data, n=100):
    Ximg = []
    ylabel = []
    for datapoint in np.random.choice(data, size=n, replace=False):
        img,rawsize = svhn.createImageData(datapoint, image_size, '../inputs/test/')
        Ximg.append(img)
        ylabel.append(datapoint['labels'])
        
    return np.array(Ximg), np.array(ylabel)
    
# change to 13068 to test on full test dataset
Ximg, ycount, ylabel = generateTestData(rawdata, 13068)

## Load models
Our training graph is composed of macro models - vision, counter and label detector. While it was suitable for training on flattened data, we will need to organize the models differently to generate sequence predictions.
Extract the models from parent graph.

In [28]:
model_yaml = open('../checkpoints/model2.yaml','r')
model = keras.models.model_from_yaml(model_yaml.read())
model_yaml.close()
model.load_weights('../checkpoints/model2.hdf5')

# enumerate the layers of main graph
for i,layer in zip(range(len(model.layers)), model.layers):
    print "layer {} : {}".format(i,layer.name)

layer 0 : train_input_img
layer 1 : vision
layer 2 : train_input_idx
layer 3 : detector


In [29]:
vision = model.layers[1]
detector = model.layers[3]

## Prediction
For prediction, we will first generate the intermediate output h, from vision model. We will then pass it to counter first. Then the detector will be called for each sample with all indices in one go. 

The crucial part here is that we want to calculate the intermediate output h only once to save on computations.

In [30]:
h = vision.predict(Ximg)

In [46]:
ylabel_ = []
for i in range(len(Ximg)):
    # generate range for each count
    indices = np.arange(max_digits)
    # one hot encoding for each index
    indices = np_utils.to_categorical(indices, max_digits)
    # tile h to match shape of indices matrix
    hs = np.tile(h[i], (max_digits,1))
    
    # predict labels for the sample
    sample_seq = detector.predict([hs, indices])
    sample_seq = np.argmax(sample_seq,1)
    ylabel_.append(sample_seq)
    
ylabel_ = [ylabelrow[ylabelrow!=0] for ylabelrow in ylabel_]
ylabel_ = [[0 if y==10 else int(y) for y in ys] for ys in ylabel_]

## Evaluation
We can evaluate the performance of both counter and label detector to get a better insight on the fit. In the end, we will evaluate the performance of the whole system. We will consider a predicted sequence to be correct only if all labels have been identified successfully, as any wrong classification can result in a totally different house number.

### Detector Metrics
Here we need to presume that counter has made a perfect prediction. If the counter itself has predicted wrong count, then it shouldn't be counted as detector's failure. So we'll evaluate the detector's performance for digits in a sequence till an index such that index is the minimum of true count value and predicted count value.

### Overall Sequence Prediction Accuracy

In [48]:
def matchSequence(seq, seq_):
    return [np.array_equal(seqi, seqi_) for seqi, seqi_ in zip(seq, seq_)]
seqmatch = matchSequence(ylabel, ylabel_)
print "Sequence prediction accuracy : {}".format(np.average(seqmatch))

Sequence prediction accuracy : 0.0


In [54]:
ylabel

array([[2, 2], [4, 10], [array(3.0)], [array(8.0)], [2, 5], [array(5.0)],
       [3, 6], [1, 10], [4, 4], [1, 2], [3, 7], [3, 3], [array(5.0)],
       [3, 4], [2, 5], [5, 8], [4, 3, 2], [4, 8], [1, 1, 7], [2, 5],
       [1, 6, 4], [1, 10], [6, 7], [3, 10], [array(9.0)], [array(6.0)],
       [2, 7], [8, 4], [2, 5], [1, 6], [2, 2], [1, 1], [1, 8, 7], [2, 3],
       [9, 2], [1, 6], [array(1.0)], [3, 3], [2, 4], [2, 2, 7], [4, 8],
       [2, 6], [8, 4], [4, 6], [4, 3], [1, 6], [array(4.0)], [1, 1, 10],
       [array(5.0)], [array(7.0)], [2, 1], [3, 5], [5, 10], [7, 7], [2, 6],
       [1, 4], [array(5.0)], [3, 6], [3, 9], [3, 10], [1, 5], [1, 8, 7],
       [array(8.0)], [2, 10], [1, 9], [5, 5], [1, 10], [2, 9, 10],
       [2, 8, 9], [1, 2, 6], [array(1.0)], [1, 5], [4, 4], [6, 3], [7, 7],
       [1, 2], [1, 1, 2], [1, 2, 7], [4, 2], [2, 4], [3, 3], [4, 6],
       [array(3.0)], [1, 2], [1, 5], [2, 4], [array(7.0)], [3, 1],
       [1, 9, 10], [array(8.0)], [8, 6], [4, 6], [4, 8], [6, 2],
    

## Display samples
Show some prediction results. The value in brackets is the true value.

In [ ]:
graphics.displaySamples(Ximg, ycounttrue=ycount, ycountpred=ycount_, ylabels=ylabel, ylabelspred=ylabel_)

## Visualize Convolutions
We can visualize the convolutions in vision model to see how the image is being processed. Change **convlayer** to see the convolutions generated by the desired layer.

In [ ]:
convlayer = 1
graphics.showCNNConv(vision, convlayer, Ximg[2])

In [ ]:
convlayer = 2
graphics.showCNNConv(vision, convlayer, Ximg[2])